In [1]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .config("spark.driver.memory","32G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.1.0")\
    .config("spark.kryoserializer.buffer.max", "1000M")\
    .getOrCreate()


:: loading settings :: url = jar:file:/home/3147567/.conda/envs/reddit_env/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/3147567/.ivy2/cache
The jars for the packages stored in: /home/3147567/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0533480-37d1-4e96-bfc6-8f5500d97b13;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.1.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

In [3]:
spark.version

'3.1.2'

In [4]:
NeutralFile = spark.read.parquet("../../Files/Submissions/score/done/Neutr_vacc_d.parquet")
ProFile = spark.read.parquet("../../Files/Submissions/score/done/Pro_vacc_d.parquet")
AntiFile = spark.read.parquet("../../Files/Submissions/score/done/Anti_vacc_d.parquet")

In [5]:
from sparknlp.base.document_assembler import DocumentAssembler
from sparknlp.base.finisher import Finisher
from sparknlp.annotator.stop_words_cleaner import StopWordsCleaner
from sparknlp.annotator.normalizer import Normalizer
from sparknlp.annotator.token import Tokenizer
from pyspark.ml.clustering import LDA

In [6]:
import functools
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [7]:
Total = unionAll([NeutralFile, ProFile, AntiFile])

In [84]:
sample_n = NeutralFile.sample(0.1)

In [15]:
sample = Total.sampleBy("class_II", fractions={
    0.0: 0.10,
    1.0: 0.10,
    2.0: 0.10
}, seed=42)

In [85]:

# remove stopwords
document_assembler = DocumentAssembler() \
    .setInputCol("cleanText") \
    .setOutputCol("document") \
    .setCleanupMode("disabled")
# Split sentence to tokens(array)
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
# clean unwanted characters and garbage
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized") \
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

finisher = Finisher() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

nlp_pipeline = Pipeline(
    stages=[
        document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,  
            finisher])

In [86]:
# train the pipeline
nlp_model = nlp_pipeline.fit(sample_n)

In [87]:
# apply the pipeline to transform dataframe.
processed_df  = nlp_model.transform(sample_n)

In [74]:
processed_df

DataFrame[cleanText: string, score: bigint, subreddit: string, created_utc: bigint, class_II: bigint, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, normalized: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, cleanTokens: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, tokens: array<string>]

In [88]:
tokens_df = processed_df.select('subreddit', 'score', 'created_utc','tokens')
tokens_df.count()

236748

In [91]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=1000, minDF=3.0)
# train the model
cv_model = cv.fit(tokens_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)

In [92]:
k=7
lda = LDA(k=k, maxIter=10, optimizer='em')
model = lda.fit(vectorized_tokens)
ll = model.logLikelihood(vectorized_tokens)
lp = model.logPerplexity(vectorized_tokens)

print( f"ANTI VACCINES, NUMBER OF TOPICS: {k}")
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))

print("The upper bound on perplexity: " + str(lp))

vocab = cv_model.vocabulary
topics = model.describeTopics(maxTermsPerTopic = 30)   

ANTI VACCINES, NUMBER OF TOPICS: 7
The lower bound on the log likelihood of the entire corpus: -11075370.045266483
The upper bound on perplexity: 6.484545368208278


In [93]:
transformed = model.transform(vectorized_tokens)

In [94]:
out = transformed[['subreddit', 'score','created_utc', 'topicDistribution']]

In [95]:
import pandas as pd
pdf = out.coalesce(1).toPandas()

In [96]:
pdf.to_pickle("../../Files/models/topics_n_7_distr.pickle")

In [55]:
topic_range = [4,5,6,7,8,9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [81]:
with open("../../Files/models/topic_an.txt", "w") as output:
    output.write(results)

In [90]:
# extract vocabulary from CountVectorizer

topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
get
time
say
work
thing
feel
good
life
need
ve
year
way
come
tell
vaccinate
death
day
right
try
die
*************************
topic: 1
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
get
time
say
work
thing
need
life
good
feel
year
ve
way
come
death
tell
vaccinate
day
right
die
try
*************************
topic: 2
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
get
time
say
work
thing
life
feel
good
ve
year
need
way
come
tell
day
vaccinate
death
try
right
die
*************************


In [126]:
model4.save('../../Files/models/topic_a_7_d.pickle')

In [8]:
from pyspark.ml.clustering import DistributedLDAModel

In [9]:
model5 = DistributedLDAModel.load('../../Files/models/topic_a_7_d.pickle')

In [15]:
topics = model5.describeTopics()

In [21]:
vocab = cv_model.vocabulary

In [19]:
topics_rdd = topics.rdd

In [22]:
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()

In [23]:
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
*************************
topic: 1
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
*************************
topic: 2
*************************
NUM
people
vaccine
like
m
know
think
covid
go
want
*************************


In [130]:
model2.save('../../Files/models/topic_a_all.pickle')

In [151]:
model3.save('../../Files/models/topic_p_n.pickle')

In [12]:
lda = LDA(k=7, maxIter=10)